# Lab 2 - Module 1: Gradient Descent on Hidden Parabola

**Learning Objectives:**
- Apply gradient descent to 1D function optimization
- Understand how learning rate affects convergence
- Compare automated GD vs. manual search from Lab 1

**Time:** ~15 minutes

---

**IMPORTANT:** Enter the same group code from Lab 1!

## Connection to Lab 1

In **Lab 1 Module 4**, you manually searched for the minimum of a hidden parabola:
- Chose x values with a slider
- Got warm/cold feedback
- Refined your guesses iteratively

**Today:** Gradient descent will do this automatically!

Same function, but now:
- GD computes the slope (gradient)
- GD moves downhill automatically
- You control the learning rate

## 1. Setup: Load Group Parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import FloatText, Button, IntText, Output, VBox, HBox
from IPython.display import display

group_code = int(input("Enter your group code: "))
np.random.seed(group_code)

# Generate same hidden function parameters as Lab 1
# Skip line parameters (we don't need them here)
_ = np.random.uniform(-3, 3)  # true_m (not used)
_ = np.random.uniform(-5, 5)  # true_b (not used)

# Hidden function parameters (parabola: f(x) = a(x-b)^2 + c)
a = np.random.uniform(0.5, 2.0)
b_param = np.random.uniform(-4, 4)
c_param = np.random.uniform(-10, 10)

def hidden_func(x):
    """The same hidden parabola from Lab 1 Module 4"""
    return a * (x - b_param)**2 + c_param

print("✓ Hidden parabola loaded (same as Lab 1 Module 4)")
print(f"\nTrue minimum is at x ≈ {b_param:.2f} with f(x) ≈ {c_param:.2f}")
print("(This is revealed now for learning purposes)")

## 2. Gradient Descent Utilities

We'll use simple numerical gradient computation (no calculus required!).

In [ ]:
def compute_gradient(x, func, h=1e-5):
    """Compute numerical gradient using central difference"""
    return (func(x + h) - func(x - h)) / (2 * h)

def gd_step(x, grad, learning_rate):
    """Single gradient descent step: new = old - lr * gradient"""
    return x - learning_rate * grad

print("✓ Gradient descent functions ready")

## 3. Prediction Question (Answer BEFORE running GD)

**Q3 (PREDICTION):** Before running the interactive tool below, predict:

Starting from x = 0.0, with five different learning rates:
- **LR = 0.01**: Will this converge quickly or slowly? Will it reach the minimum?
- **LR = 0.05**: Will this converge faster than 0.01?
- **LR = 0.4**: Will this converge faster than 0.05? Any risks?
- **LR = 1.0**: Will this be fastest? Or will it have problems?
- **LR = 3.0**: What do you expect with such a large learning rate?

Write your predictions on the answer sheet, then test them below!

## 4. Interactive Gradient Descent

**Instructions:**
1. Choose a starting point (x₀)
2. Choose a learning rate
3. Click "Run 1 Step" to see one GD update
4. Click "Run 10 Steps" to see multiple updates
5. Click "Reset" to start over with new parameters

In [ ]:
# State for interactive GD
gd_state = {
    'history': [],
    'running': False
}

# Widgets
x0_input = FloatText(description="Starting x:", value=0.0, step=0.5)
lr_input = FloatText(description="Learning rate:", value=0.1, step=0.01)
step_button = Button(description="Run 1 Step", button_style='info')
multi_step_button = Button(description="Run 10 Steps", button_style='success')
reset_button = Button(description="Reset", button_style='warning')
output_area = Output()

def initialize_gd():
    """Initialize or reset GD state"""
    x_start = x0_input.value
    gd_state['history'] = [{
        'step': 0,
        'x': x_start,
        'f(x)': hidden_func(x_start),
        'gradient': compute_gradient(x_start, hidden_func),
        'change': 0.0
    }]
    gd_state['running'] = True

def run_gd_steps(n_steps):
    """Run n steps of gradient descent"""
    if not gd_state['running']:
        initialize_gd()
    
    lr = lr_input.value
    
    for _ in range(n_steps):
        current = gd_state['history'][-1]
        x_current = current['x']
        grad = compute_gradient(x_current, hidden_func)
        
        # Update rule
        change = -lr * grad
        x_new = x_current + change
        
        gd_state['history'].append({
            'step': current['step'] + 1,
            'x': x_new,
            'f(x)': hidden_func(x_new),
            'gradient': grad,
            'change': change
        })

def update_display():
    """Update visualization and table"""
    with output_area:
        output_area.clear_output(wait=True)
        
        if not gd_state['history']:
            print("Click 'Run 1 Step' or 'Run 10 Steps' to start!")
            return
        
        # Extract history
        x_hist = [h['x'] for h in gd_state['history']]
        f_hist = [h['f(x)'] for h in gd_state['history']]
        
        # Plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)
        
        # Left: Scatter plot of GD path (no function shown)
        colors = plt.cm.viridis(np.linspace(0, 1, len(x_hist)))
        for i in range(len(x_hist)):
            marker = 'o' if i == 0 else ('*' if i == len(x_hist)-1 else 'o')
            size = 200 if i == 0 or i == len(x_hist)-1 else 100
            ax1.scatter(x_hist[i], f_hist[i], c=[colors[i]], marker=marker, 
                       s=size, edgecolors='black', linewidths=1.5, zorder=3)
        
        # Add arrows
        for i in range(len(x_hist) - 1):
            ax1.annotate('', xy=(x_hist[i+1], f_hist[i+1]), 
                        xytext=(x_hist[i], f_hist[i]),
                        arrowprops=dict(arrowstyle='->', color='red', lw=1.5, alpha=0.5))
        
        ax1.set_xlabel('x', fontsize=11)
        ax1.set_ylabel('f(x)', fontsize=11)
        ax1.set_title('Gradient Descent Path', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        
        # Right: Loss over iterations
        ax2.plot(range(len(f_hist)), f_hist, 'b-o', linewidth=2, markersize=6)
        ax2.set_xlabel('Step', fontsize=11)
        ax2.set_ylabel('f(x)', fontsize=11)
        ax2.set_title('Function Value Over Time', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Summary stats
        print(f"\nSteps taken: {len(gd_state['history']) - 1}")
        print(f"Start: x = {x_hist[0]:.4f}, f(x) = {f_hist[0]:.4f}")
        print(f"Current: x = {x_hist[-1]:.4f}, f(x) = {f_hist[-1]:.4f}")
        print(f"Improvement: {f_hist[0] - f_hist[-1]:.4f}")
        print(f"\nTrue minimum: x ≈ {b_param:.4f}, f(x) ≈ {c_param:.4f}")
        print(f"Distance from minimum: {abs(x_hist[-1] - b_param):.4f}")
        
        # Table of recent steps
        print("\n" + "="*70)
        print("Recent Steps:")
        df = pd.DataFrame(gd_state['history'][-10:])
        df_display = df[['step', 'x', 'f(x)', 'gradient', 'change']].copy()
        for col in ['x', 'f(x)', 'gradient', 'change']:
            df_display[col] = df_display[col].apply(lambda v: f"{v:.4f}")
        display(df_display)

def on_step_click(b):
    run_gd_steps(1)
    update_display()

def on_multi_step_click(b):
    run_gd_steps(10)
    update_display()

def on_reset_click(b):
    gd_state['history'] = []
    gd_state['running'] = False
    with output_area:
        output_area.clear_output()
        print("Reset! Adjust starting point and learning rate, then click 'Run 1 Step'.")

step_button.on_click(on_step_click)
multi_step_button.on_click(on_multi_step_click)
reset_button.on_click(on_reset_click)

print("Interactive Gradient Descent")
print("="*70)
print("1. Set starting point and learning rate")
print("2. Click 'Run 1 Step' to see one GD update")
print("3. Click 'Run 10 Steps' to see multiple updates")
print("4. Try different learning rates (e.g., 0.01, 0.05, 0.4, 1.0, 3.0)")
print("5. Click 'Reset' to start over\n")

display(VBox([
    HBox([x0_input, lr_input]),
    HBox([step_button, multi_step_button, reset_button]),
    output_area
]))

## 5. Comparison: Run GD with Five Learning Rates

Let's systematically compare five learning rates starting from the same point.

In [ ]:
# Compare five learning rates
x_start_compare = 0.0
learning_rates = [0.01, 0.05, 0.4, 1.0, 3.0]
max_steps = 30

results = {}

for lr in learning_rates:
    history_x = [x_start_compare]
    history_f = [hidden_func(x_start_compare)]
    
    x_current = x_start_compare
    for step in range(max_steps):
        grad = compute_gradient(x_current, hidden_func)
        x_current = gd_step(x_current, grad, lr)
        
        history_x.append(x_current)
        history_f.append(hidden_func(x_current))
    
    results[lr] = {'x': history_x, 'f': history_f}

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

colors_comparison = ['purple', 'blue', 'green', 'orange', 'red']
labels = [f'LR = {lr}' for lr in learning_rates]

# Left: Paths in (x, f(x)) space
for i, lr in enumerate(learning_rates):
    ax1.plot(results[lr]['x'], results[lr]['f'], 'o-', 
            color=colors_comparison[i], label=labels[i], alpha=0.7, linewidth=2)

ax1.axhline(y=c_param, color='black', linestyle='--', alpha=0.3, label='True minimum')
ax1.set_xlabel('x', fontsize=11)
ax1.set_ylabel('f(x)', fontsize=11)
ax1.set_title('GD Paths for Different Learning Rates', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Right: Loss over iterations
for i, lr in enumerate(learning_rates):
    ax2.plot(range(len(results[lr]['f'])), results[lr]['f'], 'o-',
            color=colors_comparison[i], label=labels[i], alpha=0.7, linewidth=2)

ax2.axhline(y=c_param, color='black', linestyle='--', alpha=0.3, label='True minimum')
ax2.set_xlabel('Step', fontsize=11)
ax2.set_ylabel('f(x)', fontsize=11)
ax2.set_title('Convergence Comparison', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\nComparison Summary:")
print("="*70)
summary_data = []
for lr in learning_rates:
    final_x = results[lr]['x'][-1]
    final_f = results[lr]['f'][-1]
    error_from_min = abs(final_x - b_param)
    summary_data.append({
        'Learning Rate': lr,
        'Final x': f"{final_x:.4f}",
        'Final f(x)': f"{final_f:.4f}",
        'Error from true min': f"{error_from_min:.4f}"
    })

display(pd.DataFrame(summary_data))

print(f"\nTrue minimum: x = {b_param:.4f}, f(x) = {c_param:.4f}")

## Questions for Your Answer Sheet

**Q4.** Compare gradient descent to your manual search in Lab 1 Module 4:
- Which was faster at finding the minimum?
- Which required more "attempts" or "steps"?
- What advantage does GD have over manual guessing?

**Q5.** Based on the visualizations above:
- How does the step size relate to: (a) the slope (gradient) magnitude, and (b) the learning rate?
- Why do the steps get smaller as you approach the minimum?
- What happens with LR = 1.0? Does it converge smoothly?
- What happens with LR = 3.0? Can you explain this behavior?

## Optional: Reveal the Full Function

In Lab 1, you never saw the underlying function. Now let's reveal it and see how GD navigated it!

In [ ]:
# Plot the actual function with GD paths overlaid
x_plot = np.linspace(-10, 10, 300)
f_plot = hidden_func(x_plot)

plt.figure(figsize=(12, 8), dpi=100)
plt.plot(x_plot, f_plot, 'b-', linewidth=2, alpha=0.3, label='Hidden function (revealed!)')

# Overlay GD paths
for i, lr in enumerate(learning_rates):
    plt.plot(results[lr]['x'], results[lr]['f'], 'o-', 
            color=colors_comparison[i], label=f'GD path (LR={lr})', 
            alpha=0.8, linewidth=2, markersize=5)

# Mark true minimum
plt.scatter([b_param], [c_param], c='red', marker='*', s=500, 
           edgecolors='black', linewidths=2, zorder=10, label='True minimum')

plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Revealed: The Hidden Parabola with GD Paths', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xlim(-10, 10)
plt.tight_layout()
plt.show()

print("This is the function you searched manually in Lab 1!")
print(f"Formula: f(x) = {a:.3f} × (x - {b_param:.3f})² + {c_param:.3f}")

## Next Steps

1. **Answer Q3, Q4, Q5** on your answer sheet
2. **Return to the LMS** and continue to Module 2
3. In Module 2, you'll apply GD to 2D parameter space (line fitting from Lab 1)!